In [1]:
import os

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from torchvision import models
import torchvision.transforms as T
from torchvision import models
from torchsummary import summary

In [2]:
from utils.utils import *
from utils.dataset_custom import SemanticSegmentationDataset
from utils.trainer_functions import train

### Перед нами стоит задача предложить модель, сегментирующую человека на фотографии 
### Вход: фотография 320x240x3. Выход: маска человека 320x240. Метрика: Dice coefficient.

In [3]:
DEVICE = torch.device('cuda')
image_size = (320,240)

#### Помимо исходного датасеты были использованы также размеченные изображения из найденного репозитория^1. Затем датасет был увеличен за счет аугментации изображений (зеркальное отображение, добавление яркости и гауссовский шум, а также вращение - находятся в отдельном пайплайне). Далее все изображения приведены к одному размеру (Прим. для моделей на базе DeepLabv3, предобученных на ImageNet, используется также нормализация входных изображений)

In [4]:
trf = T.Compose([T.Resize(image_size),
                 T.CenterCrop(image_size),
                 T.ToTensor()])
trf_rgb_imagenet = T.Compose([T.Resize(image_size),
                              T.CenterCrop(image_size),
                              T.ToTensor(),
                              T.Normalize(mean = [0.485, 0.456, 0.406],
                                          std = [0.229, 0.224, 0.225])])

In [5]:
train_data = get_paths('data/train/')+get_paths('data/augm_tr/')+get_paths('data/augm_extra_tr/')
train_mask_data = get_paths('data/train_mask/')+get_paths('data/augm_tr_mask/')+get_paths('data/augm_extra_tr_mask/')
val_data = get_paths('data/valid/')
val_mask_data = get_paths('data/valid_mask/')

In [6]:
print('Train - объектов',len(train_data))
print('Val - объектов',len(val_data))

Train - объектов 6124
Val - объектов 145


In [7]:
train_dataset_custom = SemanticSegmentationDataset(train_data,trf,train_mask_data,trf)
val_dataset_custom = SemanticSegmentationDataset(val_data,trf,val_mask_data,trf)
train_dataloader_custom = DataLoader(train_dataset_custom,batch_size = 10,shuffle=True)
val_dataloader_custom = DataLoader(val_dataset_custom,batch_size = 5)

In [8]:
train_dataset_imagenet = SemanticSegmentationDataset(train_data,trf_rgb_imagenet,train_mask_data,trf)
val_dataset_imagenet = SemanticSegmentationDataset(val_data,trf_rgb_imagenet,val_mask_data,trf)
train_dataloader_imagenet = DataLoader(train_dataset_imagenet,batch_size = 10,shuffle=True)
val_dataloader_imagenet = DataLoader(val_dataset_imagenet,batch_size = 5)

### Модель - для решения данной задачи я использовал SegNet (с нуля), кастомную версию UNET, а также предобученную модель DeeplabV3Resnet50

In [9]:
from models.custom_models import SegNet,DeepLabResnet
from models.unet import UNET_custom

In [10]:
SegnetCustom = SegNet()
#mymodel = torch.load('segnet_best.pth')
optimizer_segnet = optim.Adam(params = SegnetCustom.parameters())
#SegnetCustom = SegnetCustom.to(DEVICE)

In [11]:
DeeplabModel = DeepLabResnet()
optimizer_deeplabmodel = optim.Adam(params = DeeplabModel.parameters())
#DeeplabModel = DeeplabModel.to(DEVICE)

In [12]:
unetcustom = UNET_custom()
optimizer_unet = optim.Adam(params = unetcustom.parameters())
unetcustom = unetcustom.to(DEVICE)

### Loss Functions - приведены стандартные лосс-функции для решения задачи сегментации. Для решения проблемы  несбалансированности классов используется Focal Loss

In [13]:
def bce_loss(y_real, y_pred):
    res = y_pred - y_pred*y_real + torch.log(1+torch.exp(-y_pred))  
    return torch.mean(res)

In [14]:
def focal_loss(y_pred, y_real, alpha=0.8, gamma=2):
    y_pred = y_pred.view(-1)
    y_real = y_real.view(-1)
    BCE = bce_loss(y_pred, y_real)
    BCE_EXP = torch.exp(-BCE)
    loss = alpha * (1-BCE_EXP)**gamma * BCE

    return loss

In [15]:
def dice_loss(y_real, y_pred):      
    y_pred = y_pred.view(-1)
    y_real = y_real.view(-1)

    intersection = (y_pred * y_real).sum()                            
    dice = (2.*intersection)/(y_pred.sum() + y_real.sum())  

    return 1 - dice

### Обучение моделей

In [16]:
# train(SegnetCustom,optimizer_segnet,focal_loss,10,train_dataloader_custom,val_dataloader_custom,path_to_save= 'segnet_finalx2aug.pth')
# train(DeeplabModel,optimizer_deeplabmodel,focal_loss,10,train_dataloader_imagenet,val_dataloader_imagenet,path_to_save= 'deeplabv3_finalx2aug.pth')
# train(unetcustom,optimizer_unet,focal_loss,10,train_dataloader_custom,val_dataloader_custom,path_to_save= 'unet_finalx2aug.pth')

In [17]:
SegnetCustom = torch.load('segnet_finalx2aug.pth')
DeeplabModel = torch.load('deeplabv3_finalx2aug.pth')
UnetCustom = torch.load('unet_finalx2aug.pth')

### Final Evaluation

In [18]:
from lib.metrics import *
from utils.utils import *

In [19]:
res_segnet = list(predict(SegnetCustom,val_dataloader_custom))
res_deeplab = list(predict(DeeplabModel,val_dataloader_imagenet))
res_unet = list(predict(UnetCustom,val_dataloader_custom))
val_true = [np.array(i[1]) for i in val_dataset_custom]

#### Dice Metrics

In [20]:
get_dice(res_unet,val_true)

0.8962681481750732

In [21]:
get_dice(res_deeplab,val_true)

0.9533790059716286

In [22]:
get_dice(res_segnet,val_true)

0.923738732065178

#### IOU

In [23]:
score_model(DeeplabModel, iou_pytorch, val_dataloader_imagenet)

0.8495632615582697

In [24]:
score_model(SegnetCustom, iou_pytorch, val_dataloader_custom)

0.741560363563998

In [25]:
score_model(UnetCustom, iou_pytorch, val_dataloader_custom)

0.664548869790702

### Таким образом, модель на базе DeepLabv3Resnet50 обладает наилучшей сегментирующей способностью, поэтому используем ее для решения задачи

In [26]:
test = get_paths('data/test/')
test_dataset_imagenet = SemanticSegmentationDataset(test,transform_x = trf_rgb_imagenet,type_of_data='test')
test_dataloader = DataLoader(test_dataset_imagenet,batch_size = 5)

In [27]:
test_submission = list(predict(DeeplabModel,test_dataloader,type_of_data='test'))
test_submission_v2 = [i.astype(np.uint8) for i in test_submission]

In [28]:
from lib import *
import pandas as pd

In [29]:
rle_mask = [encode_rle(test_submission[i]) for i in range(len(test_submission))]

In [30]:
test_names = get_paths('data/test/',str_format=False)
test_index = [int(i.name.split('.')[0]) for i in test_names]

In [31]:
pd.DataFrame(data=rle_mask,index=test_index).reset_index().rename(columns={'index':'id',0:'rle_mask'}).to_csv('final_submission.csv',index=False)

In [32]:
pd.read_csv('final_submission.csv')

,id,rle_mask
0,1460,66 56 306 56 546 56 786 56 1025 57 1263 61 150...
1,1461,5660 3 5890 31 6125 42 6360 52 6597 58 6834 62...
2,1462,71 121 311 121 551 121 791 121 1031 121 1270 1...
3,1463,8234 4 8251 6 8469 12 8488 14 8555 5 8706 39 8...
4,1464,2748 2 2979 19 3215 31 3452 38 3689 45 3927 49...
...,...,...
95,1555,4195 14 4431 23 4667 43 4902 51 5139 56 5376 6...
96,1556,122 119 362 119 602 119 842 119 1082 119 1321 ...
97,1557,1 159 241 159 481 159 721 159 961 159 1201 160...
98,1558,7327 30 7562 39 7797 48 8034 54 8271 60 8508 6...


In [33]:
test_names = get_paths('data/test/')
ch = get_html(test_names, test_submission_v2, path_to_save="final_submit/pics")